## MNIST Dataset

In [34]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split

## Dataset

In [35]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [36]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [37]:
scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

## Shuffling dataset

In [38]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [39]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Outline the model

In [42]:
input_size = 784
output_size= 10
hidden_layer_size = 100

# fllaten - transforms the tensor into a vector
model = tf.keras.Sequential([
        # Flatten image into vector
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer, # 28 * 28 = 784
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # Dot product
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(output_size, activation="softmax")
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

## Training

In [44]:
NUM_EPOCHS = 6

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/6
540/540 - 8s - loss: 0.0498 - accuracy: 0.9842 - val_loss: 0.0641 - val_accuracy: 0.9828 - 8s/epoch - 15ms/step
Epoch 2/6
540/540 - 5s - loss: 0.0410 - accuracy: 0.9867 - val_loss: 0.0547 - val_accuracy: 0.9840 - 5s/epoch - 9ms/step
Epoch 3/6
540/540 - 8s - loss: 0.0351 - accuracy: 0.9891 - val_loss: 0.0479 - val_accuracy: 0.9845 - 8s/epoch - 14ms/step
Epoch 4/6
540/540 - 7s - loss: 0.0298 - accuracy: 0.9905 - val_loss: 0.0451 - val_accuracy: 0.9868 - 7s/epoch - 14ms/step
Epoch 5/6
540/540 - 7s - loss: 0.0248 - accuracy: 0.9922 - val_loss: 0.0389 - val_accuracy: 0.9878 - 7s/epoch - 13ms/step
Epoch 6/6
540/540 - 8s - loss: 0.0236 - accuracy: 0.9927 - val_loss: 0.0336 - val_accuracy: 0.9892 - 8s/epoch - 15ms/step


## Test the model

In [45]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.0755 - accuracy: 0.9770
